# AEMET Data Analysis: Availability, Correlation & Maps

Complete analysis of AEMET weather data for Balearic Islands:
1. Variable availability
2. Station-level availability
3. Correlation analysis 
4. Geographic maps

In [ ]:
import json
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import folium
import contextily as ctx

print('All imports OK')

In [ ]:
PROJECT_ROOT = Path.cwd()
AEMET_DIR = PROJECT_ROOT / 'aemet/2022'
OUTPUT_DIR = PROJECT_ROOT / 'results'
OUTPUT_DIR.mkdir(exist_ok=True)

BBOX = {'lat_min': 38.5, 'lat_max': 40.2, 'lon_min': 1.0, 'lon_max': 4.5}

# Exclude variables with no/sparse data (<10%)
EXCLUDE_VARS = [
    'psoltp', 'pliqt', 'vis', 'tss5cm', 'tss20cm',
    'stdvvu', 'geo850', 'geo700', 'stddvu', 'dvu',
    'dmaxu', 'vmaxu', 'vvu', 'pacutp', 'nieve', 'geo925'
]

AEMET_VARS = {
    'ta': 'Temperature', 'tamax': 'Temp Max', 'tamin': 'Temp Min',
    'tpr': 'Dew Point', 'ts': 'Ground Temp',
    'hr': 'Humidity', 'pres': 'Pressure', 'pres_nmar': 'Sea Lv Press',
    'prec': 'Precipitation',
    'vv': 'Wind Speed', 'vmax': 'Wind Gust', 'dv': 'Wind Dir', 'dmax': 'Gust Dir',
    'stdvv': 'Wind StdDev', 'stddv': 'Dir StdDev', 'rviento': 'Wind Run',
    'inso': 'Insolation',
}

# Variable labels for plotting
VAR_LABELS = {
    'ta': 'Temperature (°C)', 'tamax': 'Temp Max (°C)', 'tamin': 'Temp Min (°C)',
    'tpr': 'Dew Point (°C)', 'ts': 'Ground Temp (°C)',
    'hr': 'Humidity (%)', 'pres': 'Pressure (hPa)', 'pres_nmar': 'Sea Lv Press (hPa)',
    'prec': 'Precipitation (mm)',
    'vv': 'Wind Speed (m/s)', 'vmax': 'Wind Gust (m/s)', 'dv': 'Wind Dir (°)', 'dmax': 'Gust Dir (°)',
    'stdvv': 'Wind StdDev', 'stddv': 'Dir StdDev', 'rviento': 'Wind Run',
    'inso': 'Insolation (h)',
}

# Island boundaries
ISLANDS = {
    'Mallorca': {'lon_min': 2.3, 'lon_max': 3.5, 'lat_min': 39.2, 'lat_max': 39.95},
    'Menorca': {'lon_min': 3.8, 'lon_max': 4.35, 'lat_min': 39.8, 'lat_max': 40.1},
    'Ibiza': {'lon_min': 1.15, 'lon_max': 1.65, 'lat_min': 38.85, 'lat_max': 39.15},
}


In [ ]:
# Load data
json_files = sorted(glob.glob(str(AEMET_DIR / '*.json')))
all_records = []
for f in json_files:
    try:
        with open(f, 'r', encoding='utf-8') as file:
            data = json.loads(file.read())
            if 'datos' in data:
                all_records.extend(data['datos'])
    except:
        pass

df = pd.DataFrame(all_records)
df['fint'] = pd.to_datetime(df['fint'])

# Filter Balearic
df_bal = df[
    (df['lat'] >= BBOX['lat_min']) & (df['lat'] <= BBOX['lat_max']) &
    (df['lon'] >= BBOX['lon_min']) & (df['lon'] <= BBOX['lon_max'])
].copy()

print(f'Total records: {len(df_bal):,}')
print(f'Stations: {df_bal["idema"].nunique()}')

## 1. Variable Availability

In [ ]:
available_vars = [v for v in AEMET_VARS.keys() if v in df_bal.columns and v not in EXCLUDE_VARS]

availability = []
for var in available_vars:
    total = len(df_bal)
    valid = df_bal[var].notna().sum()
    pct = (valid / total * 100)
    availability.append({'var': var, 'label': AEMET_VARS[var], 'valid_count': valid,
                         'total_count': total, 'availability_pct': round(pct, 1)})

avail_df = pd.DataFrame(availability).sort_values('availability_pct', ascending=False).reset_index(drop=True)

print('VARIABLE AVAILABILITY')
print('=' * 65)
for _, row in avail_df.iterrows():
    status = 'GOOD' if row['availability_pct'] >= 80 else 'PARTIAL' if row['availability_pct'] >= 40 else 'SPARSE'
    print(f'{row["var"]:<10} {row["label"]:<15} {row["availability_pct"]:>6.1f}%  [{status}]')

print(f'\nExcluded (no data): {EXCLUDE_VARS}')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
colors = ['#2ecc71' if p >= 80 else '#f39c12' if p >= 40 else '#e74c3c' for p in avail_df['availability_pct']]
bars = ax.barh(range(len(avail_df)), avail_df['availability_pct'], color=colors, edgecolor='black')
ax.set_yticks(range(len(avail_df)))
ax.set_yticklabels([f"{r['var']} ({r['label']})" for _, r in avail_df.iterrows()])
ax.axvline(80, color='green', ls='--', alpha=0.7, label='Good (80%)')
ax.axvline(40, color='orange', ls='--', alpha=0.7, label='Partial (40%)')
ax.set_xlim(0, 105)
ax.set_xlabel('Availability %')
ax.set_title('Variable Availability', fontweight='bold')
ax.legend(loc='lower right')
for i, pct in enumerate(avail_df['availability_pct']):
    ax.text(pct + 1, i, f'{pct:.1f}%', va='center', fontsize=9)
ax.invert_yaxis()
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'variable_availability.png', dpi=150)
plt.show()

## 2. Availability per Station

In [ ]:
stations = df_bal.groupby('idema').agg({
    'lat': 'first', 'lon': 'first', 'alt': 'first', 'ubi': 'first', 'fint': 'count'
}).reset_index()
stations.columns = ['idema', 'lat', 'lon', 'alt', 'name', 'n_records']

for var in available_vars:
    counts = df_bal.groupby('idema')[var].apply(lambda x: x.notna().sum()).reset_index()
    counts.columns = ['idema', f'{var}_count']
    stations = stations.merge(counts, on='idema', how='left')
    stations[f'{var}_pct'] = (stations[f'{var}_count'] / stations['n_records'] * 100).round(1)

pct_cols = [f'{v}_pct' for v in available_vars]
stations['overall_pct'] = stations[pct_cols].mean(axis=1).round(1)

print(f'Stations: {len(stations)}')

In [ ]:
# Heatmap
heatmap_df = stations.sort_values('overall_pct', ascending=False).copy()
heatmap_df['label'] = heatmap_df['name'].str[:22] + ' (' + heatmap_df['alt'].astype(int).astype(str) + 'm)'
heatmap_data = heatmap_df.set_index('label')[pct_cols]
heatmap_data.columns = [AEMET_VARS[c.replace('_pct', '')] for c in heatmap_data.columns]

fig, ax = plt.subplots(figsize=(14, max(10, len(heatmap_data) * 0.35)))
sns.heatmap(heatmap_data, annot=True, fmt='.0f', cmap='RdYlGn', vmin=0, vmax=100, ax=ax,
            cbar_kws={'label': 'Availability %'}, annot_kws={'size': 8})
ax.set_title('Availability by Station and Variable (%)', fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'station_variable_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Correlation Analysis 

**Note:** High correlation shown for information.

In [ ]:
corr_matrix = df_bal[available_vars].corr()

fig, ax = plt.subplots(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, vmin=-1, vmax=1, square=True, ax=ax, annot_kws={'size': 9})
ax.set_title('Variable Correlation Matrix', fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'correlation_matrix.png', dpi=150)
plt.show()

In [ ]:
# Find and show highly correlated pairs (informative only)
print('HIGHLY CORRELATED PAIRS (|r| >= 0.9)')
print('=' * 80)
print('Note: For information only. Decide based on your model needs.')
print('-' * 80)

high_corr_pairs = []
for i, var1 in enumerate(available_vars):
    for j, var2 in enumerate(available_vars):
        if i < j:
            corr = corr_matrix.loc[var1, var2]
            if abs(corr) >= 0.9:
                a1 = avail_df[avail_df['var'] == var1]['availability_pct'].values[0]
                a2 = avail_df[avail_df['var'] == var2]['availability_pct'].values[0]
                high_corr_pairs.append({'var1': var1, 'var2': var2, 'corr': round(corr, 3),
                                       'avail1': a1, 'avail2': a2})
                print(f'{var1:<10} ~ {var2:<10}  r = {corr:>6.3f}  (avail: {a1:.0f}% vs {a2:.0f}%)')

if not high_corr_pairs:
    print('No pairs with |r| >= 0.9 found')

high_corr_df = pd.DataFrame(high_corr_pairs)

In [ ]:
# Summary table: Variable, Availability, Correlations
print('\nVARIABLE SUMMARY')
print('=' * 90)
print(f'{"Variable":<10} {"Description":<15} {"Availability":>12} {"Correlated with (r>=0.9)":<40}')
print('-' * 90)

for _, row in avail_df.iterrows():
    var = row['var']
    corr_with = []
    for _, p in high_corr_df.iterrows() if len(high_corr_df) > 0 else []:
        if p['var1'] == var:
            corr_with.append(f"{p['var2']}({p['corr']:.2f})")
        elif p['var2'] == var:
            corr_with.append(f"{p['var1']}({p['corr']:.2f})")
    corr_str = ', '.join(corr_with) if corr_with else '-'
    print(f'{var:<10} {row["label"]:<15} {row["availability_pct"]:>11.1f}%  {corr_str:<40}')

## 4. Geographic Maps

In [ ]:
### Interactive Map - Data Availability
m_avail = folium.Map(location=[stations['lat'].mean(), stations['lon'].mean()], zoom_start=8, tiles=None)

folium.TileLayer('https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
                 attr='Esri', name='Topographic').add_to(m_avail)
folium.TileLayer('OpenStreetMap', name='OpenStreetMap').add_to(m_avail)

for _, row in stations.iterrows():
    pct = row['overall_pct']
    color = '#2ecc71' if pct >= 80 else '#f39c12' if pct >= 50 else '#e74c3c'
    popup = f'<b>{row["name"]}</b><br>ID: {row["idema"]}<br>Alt: {row["alt"]:.0f}m<br>Avail: {pct:.0f}%'
    folium.CircleMarker([row['lat'], row['lon']], radius=10, color='black', weight=2,
                        fill=True, fillColor=color, fillOpacity=0.85,
                        popup=folium.Popup(popup, max_width=250)).add_to(m_avail)

folium.LayerControl().add_to(m_avail)
legend = '''<div style="position:fixed;bottom:50px;left:50px;z-index:1000;background:white;padding:10px;border:2px solid #333;border-radius:5px">
<b>Availability</b><br>
<i style="background:#2ecc71;width:12px;height:12px;display:inline-block;border-radius:50%"></i> ≥80%<br>
<i style="background:#f39c12;width:12px;height:12px;display:inline-block;border-radius:50%"></i> 50-80%<br>
<i style="background:#e74c3c;width:12px;height:12px;display:inline-block;border-radius:50%"></i> <50%</div>'''
m_avail.get_root().html.add_child(folium.Element(legend))
m_avail.save(str(OUTPUT_DIR / 'map_availability.html'))
print('Saved: map_availability.html')
m_avail

In [ ]:
# Interactive Maps - Insolation and Pressure Availability

# Calculate availability per station for specific variables
def calc_var_availability(df, stations_df, var):
    avail = []
    for _, st in stations_df.iterrows():
        st_data = df[df['idema'] == st['idema']]
        total = len(st_data)
        valid = st_data[var].notna().sum() if var in st_data.columns else 0
        pct = (valid / total * 100) if total > 0 else 0
        avail.append({
            'idema': st['idema'],
            'name': st['name'],
            'lat': st['lat'],
            'lon': st['lon'],
            'alt': st['alt'],
            f'{var}_pct': pct,
            f'{var}_count': valid,
            'total': total
        })
    return pd.DataFrame(avail)

# Calculate availability for inso and pres
inso_avail = calc_var_availability(df_bal, stations, 'inso')
pres_avail = calc_var_availability(df_bal, stations, 'pres')

print('VARIABLE AVAILABILITY BY STATION')
print('=' * 70)
print(f'{"Variable":<15} {"Stations with data":<20} {"Mean availability":<20}')
print('-' * 70)
print(f'{"Insolation":<15} {(inso_avail["inso_pct"] > 0).sum():<20} {inso_avail["inso_pct"].mean():.1f}%')
print(f'{"Pressure":<15} {(pres_avail["pres_pct"] > 0).sum():<20} {pres_avail["pres_pct"].mean():.1f}%')

In [ ]:
# Map 1: Insolation (inso) availability
m_inso = folium.Map(location=[stations['lat'].mean(), stations['lon'].mean()], zoom_start=8, tiles=None)

folium.TileLayer('https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
                 attr='Esri', name='Topographic').add_to(m_inso)
folium.TileLayer('OpenStreetMap', name='OpenStreetMap').add_to(m_inso)

for _, row in inso_avail.iterrows():
    pct = row['inso_pct']
    
    if pct >= 80:
        color = '#2ecc71'  # Green
    elif pct >= 50:
        color = '#f39c12'  # Orange
    elif pct > 0:
        color = '#e74c3c'  # Red
    else:
        color = '#95a5a6'  # Gray - no data
    
    popup = f'''
    <b>{row["name"]}</b><br>
    ID: {row["idema"]}<br>
    Alt: {row["alt"]:.0f}m<br>
    <hr>
    <b>Insolation (inso)</b><br>
    Availability: {pct:.1f}%<br>
    Records: {row["inso_count"]:,.0f} / {row["total"]:,.0f}
    '''
    
    # Size based on availability
    radius = 12 if pct >= 50 else 8 if pct > 0 else 6
    
    folium.CircleMarker(
        [row['lat'], row['lon']], 
        radius=radius, 
        color='black', 
        weight=2,
        fill=True, 
        fillColor=color, 
        fillOpacity=0.85,
        popup=folium.Popup(popup, max_width=250)
    ).add_to(m_inso)

folium.LayerControl().add_to(m_inso)

legend_inso = '''
<div style="position:fixed;bottom:50px;left:50px;z-index:1000;background:white;padding:10px;border:2px solid #333;border-radius:5px">
<b>Insolation (inso)</b><br>
<i style="background:#2ecc71;width:12px;height:12px;display:inline-block;border-radius:50%"></i> ≥80%<br>
<i style="background:#f39c12;width:12px;height:12px;display:inline-block;border-radius:50%"></i> 50-80%<br>
<i style="background:#e74c3c;width:12px;height:12px;display:inline-block;border-radius:50%"></i> 1-50%<br>
<i style="background:#95a5a6;width:12px;height:12px;display:inline-block;border-radius:50%"></i> No data
</div>
'''
m_inso.get_root().html.add_child(folium.Element(legend_inso))

m_inso.save(str(OUTPUT_DIR / 'map_insolation_availability.html'))
print('✅ Saved: map_insolation_availability.html')
m_inso

In [ ]:
# Map 2: Pressure (pres) availability
m_pres = folium.Map(location=[stations['lat'].mean(), stations['lon'].mean()], zoom_start=8, tiles=None)

folium.TileLayer('https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
                 attr='Esri', name='Topographic').add_to(m_pres)
folium.TileLayer('OpenStreetMap', name='OpenStreetMap').add_to(m_pres)

for _, row in pres_avail.iterrows():
    pct = row['pres_pct']
    
    if pct >= 80:
        color = '#3498db'  # Blue
    elif pct >= 50:
        color = '#9b59b6'  # Purple
    elif pct > 0:
        color = '#e74c3c'  # Red
    else:
        color = '#95a5a6'  # Gray - no data
    
    popup = f'''
    <b>{row["name"]}</b><br>
    ID: {row["idema"]}<br>
    Alt: {row["alt"]:.0f}m<br>
    <hr>
    <b>Pressure (pres)</b><br>
    Availability: {pct:.1f}%<br>
    Records: {row["pres_count"]:,.0f} / {row["total"]:,.0f}
    '''
    
    radius = 12 if pct >= 50 else 8 if pct > 0 else 6
    
    folium.CircleMarker(
        [row['lat'], row['lon']], 
        radius=radius, 
        color='black', 
        weight=2,
        fill=True, 
        fillColor=color, 
        fillOpacity=0.85,
        popup=folium.Popup(popup, max_width=250)
    ).add_to(m_pres)

folium.LayerControl().add_to(m_pres)

legend_pres = '''
<div style="position:fixed;bottom:50px;left:50px;z-index:1000;background:white;padding:10px;border:2px solid #333;border-radius:5px">
<b>Pressure (pres)</b><br>
<i style="background:#3498db;width:12px;height:12px;display:inline-block;border-radius:50%"></i> ≥80%<br>
<i style="background:#9b59b6;width:12px;height:12px;display:inline-block;border-radius:50%"></i> 50-80%<br>
<i style="background:#e74c3c;width:12px;height:12px;display:inline-block;border-radius:50%"></i> 1-50%<br>
<i style="background:#95a5a6;width:12px;height:12px;display:inline-block;border-radius:50%"></i> No data
</div>
'''
m_pres.get_root().html.add_child(folium.Element(legend_pres))

m_pres.save(str(OUTPUT_DIR / 'map_pressure_availability.html'))
print('✅ Saved: map_pressure_availability.html')
m_pres

In [ ]:
# Summary table: Stations with insolation and pressure data
print('\nSTATIONS WITH INSOLATION DATA')
print('=' * 70)
inso_stations = inso_avail[inso_avail['inso_pct'] > 0].sort_values('inso_pct', ascending=False)
print(f'{"Station":<35} {"Alt (m)":<10} {"Avail %":<10} {"Records":<15}')
print('-' * 70)
for _, row in inso_stations.iterrows():
    print(f'{row["name"][:33]:<35} {row["alt"]:<10.0f} {row["inso_pct"]:<10.1f} {row["inso_count"]:,.0f}')

print(f'\nTotal stations with inso: {len(inso_stations)}')

print('\n' + '=' * 70)
print('STATIONS WITH PRESSURE DATA')
print('=' * 70)
pres_stations = pres_avail[pres_avail['pres_pct'] > 0].sort_values('pres_pct', ascending=False)
print(f'{"Station":<35} {"Alt (m)":<10} {"Avail %":<10} {"Records":<15}')
print('-' * 70)
for _, row in pres_stations.iterrows():
    print(f'{row["name"][:33]:<35} {row["alt"]:<10.0f} {row["pres_pct"]:<10.1f} {row["pres_count"]:,.0f}')

print(f'\nTotal stations with pres: {len(pres_stations)}')

In [ ]:
# STATION-TO-STATION VARIANCE ANALYSIS - PER ISLAND
# Compare stations WITHIN each island only (not between islands)

print('STATION-TO-STATION VARIANCE ANALYSIS - PER ISLAND')
print('=' * 80)
print('Comparing measurements between stations WITHIN the same island')
print('=' * 80)

from itertools import combinations
from haversine import haversine, Unit

# Variables to analyze
vars_to_analyze = ['ta', 'hr', 'prec', 'vv', 'pres', 'inso']
vars_to_analyze = [v for v in vars_to_analyze if v in df_bal.columns]

VAR_LABELS = {
    'ta': 'Temperature (°C)', 'hr': 'Humidity (%)', 'prec': 'Precipitation (mm)',
    'vv': 'Wind Speed (m/s)', 'pres': 'Pressure (hPa)', 'inso': 'Insolation (h)'
}

ISLANDS = {
    'Mallorca': {'lon_min': 2.3, 'lon_max': 3.5, 'lat_min': 39.2, 'lat_max': 39.95},
    'Menorca': {'lon_min': 3.8, 'lon_max': 4.35, 'lat_min': 39.8, 'lat_max': 40.1},
    'Ibiza': {'lon_min': 1.15, 'lon_max': 1.65, 'lat_min': 38.85, 'lat_max': 39.15},
}

# Prepare data with hourly resolution
df_bal['hour'] = df_bal['fint'].dt.floor('h')

# Get station info with island
station_info = df_bal.groupby('idema').agg({
    'lat': 'first',
    'lon': 'first', 
    'ubi': 'first',
    'alt': 'first'
}).reset_index()
station_info.rename(columns={'ubi': 'name'}, inplace=True)

# Assign island to each station
def detect_island(row):
    for island, bbox in ISLANDS.items():
        if bbox['lon_min'] <= row['lon'] <= bbox['lon_max'] and bbox['lat_min'] <= row['lat'] <= bbox['lat_max']:
            return island
    return 'Unknown'

station_info['island'] = station_info.apply(detect_island, axis=1)

print(f'\nStations per island:')
for island in ISLANDS.keys():
    n = len(station_info[station_info['island'] == island])
    print(f'  {island}: {n} stations')

print(f'\nVariables: {vars_to_analyze}')

In [ ]:
# Calculate pairwise differences for stations WITHIN each island
print('\nCalculating pairwise station differences PER ISLAND...')

pairwise_results = []

for island in ISLANDS.keys():
    island_stations = station_info[station_info['island'] == island]['idema'].tolist()
    
    if len(island_stations) < 2:
        print(f'  {island}: Not enough stations ({len(island_stations)})')
        continue
    
    station_pairs = list(combinations(island_stations, 2))
    print(f'  {island}: {len(island_stations)} stations → {len(station_pairs)} pairs')
    
    for st1, st2 in station_pairs:
        # Get station info
        info1 = station_info[station_info['idema'] == st1].iloc[0]
        info2 = station_info[station_info['idema'] == st2].iloc[0]
        
        # Calculate distance
        dist = haversine((info1['lat'], info1['lon']), (info2['lat'], info2['lon']), unit=Unit.KILOMETERS)
        alt_diff = abs(info1['alt'] - info2['alt'])
        
        # Get data for both stations
        data1 = df_bal[df_bal['idema'] == st1][['hour'] + vars_to_analyze].groupby('hour').mean()
        data2 = df_bal[df_bal['idema'] == st2][['hour'] + vars_to_analyze].groupby('hour').mean()
        
        # Merge on hour (same timestamp)
        merged = data1.join(data2, lsuffix='_1', rsuffix='_2', how='inner')
        
        if len(merged) < 10:
            continue
        
        pair_result = {
            'island': island,
            'station1': info1['name'],
            'station2': info2['name'],
            'idema1': st1,
            'idema2': st2,
            'distance_km': dist,
            'alt_diff_m': alt_diff,
            'n_common_hours': len(merged)
        }
        
        # Calculate statistics for each variable
        for var in vars_to_analyze:
            col1, col2 = f'{var}_1', f'{var}_2'
            if col1 in merged.columns and col2 in merged.columns:
                mask = merged[[col1, col2]].notna().all(axis=1)
                if mask.sum() < 10:
                    continue
                
                v1 = merged.loc[mask, col1].values
                v2 = merged.loc[mask, col2].values
                
                diff = v1 - v2
                
                pair_result[f'{var}_mean_diff'] = diff.mean()
                pair_result[f'{var}_std_diff'] = diff.std()
                pair_result[f'{var}_mae'] = np.abs(diff).mean()
                pair_result[f'{var}_max_diff'] = np.abs(diff).max()
                pair_result[f'{var}_correlation'] = np.corrcoef(v1, v2)[0, 1]
                pair_result[f'{var}_n_obs'] = mask.sum()
        
        pairwise_results.append(pair_result)

pairs_df = pd.DataFrame(pairwise_results)
print(f'\nTotal valid pairs: {len(pairs_df)}')
print(pairs_df.groupby('island').size())

In [ ]:
# Summary statistics by variable AND island
print('\n' + '=' * 100)
print('PAIRWISE DIFFERENCE SUMMARY BY ISLAND')
print('=' * 100)

island_var_summary = []

for island in ISLANDS.keys():
    island_pairs = pairs_df[pairs_df['island'] == island]
    
    if len(island_pairs) == 0:
        continue
    
    print(f'\n{island.upper()} ({len(island_pairs)} pairs)')
    print('-' * 90)
    print(f'{"Variable":<20} {"Mean Diff":<12} {"Std Diff":<12} {"MAE":<12} {"Max Diff":<12} {"Corr":<10}')
    print('-' * 90)
    
    for var in vars_to_analyze:
        cols = [f'{var}_mean_diff', f'{var}_std_diff', f'{var}_mae', f'{var}_max_diff', f'{var}_correlation']
        if not all(c in island_pairs.columns for c in cols):
            continue
        
        valid = island_pairs[island_pairs[f'{var}_mae'].notna()]
        if len(valid) == 0:
            continue
        
        mean_diff = valid[f'{var}_mean_diff'].mean()
        std_diff = valid[f'{var}_std_diff'].mean()
        mae = valid[f'{var}_mae'].mean()
        max_diff = valid[f'{var}_max_diff'].mean()
        mean_corr = valid[f'{var}_correlation'].mean()
        
        island_var_summary.append({
            'island': island,
            'variable': var,
            'label': VAR_LABELS.get(var, var),
            'n_pairs': len(valid),
            'mean_diff': mean_diff,
            'std_diff': std_diff,
            'mae': mae,
            'max_diff': max_diff,
            'correlation': mean_corr
        })
        
        print(f'{VAR_LABELS.get(var, var):<20} {mean_diff:>+11.3f} {std_diff:>11.3f} {mae:>11.3f} {max_diff:>11.3f} {mean_corr:>9.3f}')

island_summary_df = pd.DataFrame(island_var_summary)

In [ ]:
# Visualize MAE distribution per island
fig, axes = plt.subplots(len(ISLANDS), len(vars_to_analyze[:6]), figsize=(18, 4 * len(ISLANDS)))

for i, island in enumerate(ISLANDS.keys()):
    island_pairs = pairs_df[pairs_df['island'] == island]
    
    for j, var in enumerate(vars_to_analyze[:6]):
        ax = axes[i, j] if len(ISLANDS) > 1 else axes[j]
        
        mae_col = f'{var}_mae'
        if mae_col not in island_pairs.columns or island_pairs[mae_col].isna().all():
            ax.text(0.5, 0.5, 'No data', ha='center', va='center', transform=ax.transAxes)
            ax.set_title(f'{island}\n{VAR_LABELS.get(var, var)[:15]}', fontsize=10)
            continue
        
        mae_values = island_pairs[mae_col].dropna()
        
        if len(mae_values) > 0:
            ax.hist(mae_values, bins=min(20, len(mae_values)), color='steelblue', alpha=0.7, edgecolor='black')
            ax.axvline(x=mae_values.mean(), color='red', linestyle='--', linewidth=2)
            ax.set_xlabel('MAE')
            ax.set_ylabel('Pairs')
            ax.set_title(f'{island}\n{VAR_LABELS.get(var, var)[:15]}\nMAE={mae_values.mean():.2f}', fontsize=10)
        else:
            ax.text(0.5, 0.5, 'No data', ha='center', va='center', transform=ax.transAxes)
            ax.set_title(f'{island}\n{VAR_LABELS.get(var, var)[:15]}', fontsize=10)

plt.suptitle('MAE Distribution Between Station Pairs (Within Island)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'pairwise_mae_per_island.png', dpi=150)
plt.show()

In [ ]:
# Correlation vs Distance per island
fig, axes = plt.subplots(len(ISLANDS), len(vars_to_analyze[:6]), figsize=(18, 4 * len(ISLANDS)))

for i, island in enumerate(ISLANDS.keys()):
    island_pairs = pairs_df[pairs_df['island'] == island]
    
    for j, var in enumerate(vars_to_analyze[:6]):
        ax = axes[i, j] if len(ISLANDS) > 1 else axes[j]
        
        corr_col = f'{var}_correlation'
        if corr_col not in island_pairs.columns:
            ax.set_visible(False)
            continue
        
        valid = island_pairs[['distance_km', corr_col]].dropna()
        
        if len(valid) < 3:
            ax.text(0.5, 0.5, f'Not enough data\n({len(valid)} pairs)', 
                   ha='center', va='center', transform=ax.transAxes)
            ax.set_title(f'{island} - {VAR_LABELS.get(var, var)[:12]}', fontsize=10)
            continue
        
        ax.scatter(valid['distance_km'], valid[corr_col], alpha=0.6, s=40, c='steelblue')
        
        # Trend line
        if len(valid) > 3:
            z = np.polyfit(valid['distance_km'], valid[corr_col], 1)
            p = np.poly1d(z)
            x_line = np.linspace(valid['distance_km'].min(), valid['distance_km'].max(), 50)
            ax.plot(x_line, p(x_line), 'r-', linewidth=2)
            ax.text(0.95, 0.05, f'slope={z[0]:.4f}', transform=ax.transAxes, 
                   ha='right', fontsize=8, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        ax.axhline(y=0.8, color='green', linestyle=':', alpha=0.5)
        ax.set_xlabel('Distance (km)')
        ax.set_ylabel('Correlation')
        ax.set_title(f'{island} - {VAR_LABELS.get(var, var)[:12]}', fontsize=10)
        ax.set_ylim(-0.2, 1.05)
        ax.grid(True, alpha=0.3)

plt.suptitle('Station Correlation vs Distance (Within Island)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'correlation_vs_distance_per_island.png', dpi=150)
plt.show()

In [ ]:
# Compare islands side by side
print('\n' + '=' * 100)
print('ISLAND COMPARISON: SPATIAL VARIANCE')
print('=' * 100)

# Pivot table for comparison
if len(island_summary_df) > 0:
    for metric in ['mae', 'correlation']:
        print(f'\n{metric.upper()}:')
        pivot = island_summary_df.pivot(index='label', columns='island', values=metric)
        print(pivot.round(3).to_string())

# Bar chart comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# MAE comparison
ax1 = axes[0]
if len(island_summary_df) > 0:
    pivot_mae = island_summary_df.pivot(index='label', columns='island', values='mae')
    pivot_mae.plot(kind='bar', ax=ax1, alpha=0.8, edgecolor='black')
    ax1.set_ylabel('Mean Absolute Error')
    ax1.set_title('MAE Between Stations by Island', fontweight='bold')
    ax1.legend(title='Island')
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(True, alpha=0.3, axis='y')

# Correlation comparison
ax2 = axes[1]
if len(island_summary_df) > 0:
    pivot_corr = island_summary_df.pivot(index='label', columns='island', values='correlation')
    pivot_corr.plot(kind='bar', ax=ax2, alpha=0.8, edgecolor='black')
    ax2.set_ylabel('Mean Correlation')
    ax2.set_title('Station Correlation by Island', fontweight='bold')
    ax2.legend(title='Island')
    ax2.tick_params(axis='x', rotation=45)
    ax2.axhline(y=0.8, color='green', linestyle='--', alpha=0.5, label='Good threshold')
    ax2.grid(True, alpha=0.3, axis='y')
    ax2.set_ylim(0, 1.05)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'island_variance_comparison.png', dpi=150)
plt.show()

In [ ]:
# SPATIAL vs TEMPORAL VARIABILITY ANALYSIS (AEMET Stations)
# Analyze if sunshine and pressure vary more by LOCATION or by TIME

print("SPATIAL vs TEMPORAL VARIABILITY ANALYSIS (AEMET)")
print("=" * 70)
print("Question: Do insolation and pressure vary more by LOCATION or by TIME?")
print("If temporal >> spatial → fewer stations needed")
print("=" * 70)

variables = ['inso', 'pres']
var_labels = {'inso': 'Insolation (h)', 'pres': 'Pressure (hPa)'}

# Check which variables are available
variables = [v for v in variables if v in df_bal.columns]
if not variables:
    print("⚠️ Required variables not found in df_bal")
    print(f"Available columns: {df_bal.columns.tolist()}")
else:
    # Add hour column if not present
    if 'hour' not in df_bal.columns:
        df_bal['hour'] = df_bal['fint'].dt.floor('h')
    
    # Assign island to stations if not done
    if 'island' not in stations.columns:
        def detect_island(row):
            for island, bbox in ISLANDS.items():
                if bbox['lon_min'] <= row['lon'] <= bbox['lon_max'] and bbox['lat_min'] <= row['lat'] <= bbox['lat_max']:
                    return island
            return 'Unknown'
        stations['island'] = stations.apply(detect_island, axis=1)
    
    # Merge island info to df_bal
    if 'island' not in df_bal.columns:
        df_bal = df_bal.merge(stations[['idema', 'island']], on='idema', how='left')
    
    results = []
    
    for island in ISLANDS.keys():
        island_stations = stations[stations['island'] == island]['idema'].tolist()
        island_data = df_bal[df_bal['idema'].isin(island_stations)]
        n_stations = len(island_stations)
        
        if n_stations < 2:
            print(f"⚠️  {island}: Only {n_stations} station(s) - skipping")
            continue
        
        for var in variables:
            if var not in island_data.columns:
                continue
            
            # Only use data where variable is available
            var_data = island_data[island_data[var].notna()]
            
            if len(var_data) < 100:
                print(f"⚠️  {island}/{var}: Only {len(var_data)} records - skipping")
                continue
            
            # Spatial std: variation across stations at same time
            spatial_std = var_data.groupby('hour')[var].std().mean()
            
            # Temporal std: variation over time at same station
            temporal_std = var_data.groupby('idema')[var].std().mean()
            
            # Spatial range: max - min across stations at same time
            spatial_range = var_data.groupby('hour')[var].apply(lambda x: x.max() - x.min()).mean()
            
            # Number of stations with this variable
            n_with_var = var_data['idema'].nunique()
            
            ratio = temporal_std / spatial_std if spatial_std > 0 else np.inf
            
            results.append({
                'island': island,
                'variable': var_labels.get(var, var),
                'var_code': var,
                'n_stations': n_with_var,
                'spatial_std': spatial_std,
                'spatial_range': spatial_range,
                'temporal_std': temporal_std,
                'ratio': ratio
            })
    
    if len(results) == 0:
        print("⚠️ No valid results - check data availability for inso and pres")
    else:
        results_df = pd.DataFrame(results)
        
        print(f"\n{'Island':<12} {'Variable':<15} {'Stations':>8} {'Spatial σ':>10} {'Range':>10} {'Temporal σ':>11} {'Ratio':>8}")
        print("-" * 80)
        for _, row in results_df.iterrows():
            print(f"{row['island']:<12} {row['variable']:<15} {row['n_stations']:>8} {row['spatial_std']:>10.2f} "
                  f"{row['spatial_range']:>10.2f} {row['temporal_std']:>11.2f} {row['ratio']:>7.1f}x")
        print("-" * 80)
        print("\nINTERPRETATION:")
        print("  Ratio > 5  → ONE station sufficient (time dominates)")
        print("  Ratio 2-5  → FEW stations enough")
        print("  Ratio < 2  → MULTIPLE stations needed (location matters)")
        
        # Visualization
        available_vars = results_df['var_code'].unique()
        n_vars = len(available_vars)
        n_islands = len(ISLANDS)
        
        if n_vars > 0:
            fig, axes = plt.subplots(n_vars, n_islands, figsize=(14, 4 * n_vars))
            if n_vars == 1:
                axes = axes.reshape(1, -1)
            
            for col, island in enumerate(ISLANDS.keys()):
                island_stations = stations[stations['island'] == island]['idema'].tolist()
                island_data = df_bal[df_bal['idema'].isin(island_stations)]
                
                for row, var in enumerate(available_vars):
                    ax = axes[row, col] if n_vars > 1 else axes[col]
                    label = var_labels.get(var, var)
                    
                    if var not in island_data.columns:
                        ax.text(0.5, 0.5, f'No {var} data', ha='center', va='center', transform=ax.transAxes)
                        continue
                    
                    var_data = island_data[island_data[var].notna()]
                    
                    if len(var_data) < 10:
                        ax.text(0.5, 0.5, f'Insufficient {var} data', ha='center', va='center', transform=ax.transAxes)
                        continue
                    
                    hourly_stats = var_data.groupby('hour')[var].agg(['mean', 'std', 'min', 'max']).reset_index()
                    hourly_stats = hourly_stats.sort_values('hour').reset_index(drop=True)
                    
                    ax.fill_between(range(len(hourly_stats)), hourly_stats['min'], hourly_stats['max'], 
                                    alpha=0.3, color='blue', label='Spatial range')
                    ax.plot(hourly_stats['mean'], 'b-', linewidth=1.5, label='Island mean')
                    
                    ax.set_title(f"{island}\n{label} ({var_data['idema'].nunique()} stations)", fontsize=10, fontweight='bold')
                    ax.set_xlabel('Hour index')
                    
                    if col == 0:
                        ax.set_ylabel(label.split('(')[0].strip())
                    if row == 0 and col == n_islands - 1:
                        ax.legend(loc='upper right', fontsize=8)
                    
                    ax.grid(True, alpha=0.3)
            
            plt.suptitle('AEMET: Temporal Evolution with Spatial Range\n(narrow bands = low spatial variance = fewer stations needed)', 
                         fontsize=12, fontweight='bold', y=1.02)
            plt.tight_layout()
            plt.savefig(OUTPUT_DIR / 'aemet_spatial_temporal_variability.png', dpi=150, bbox_inches='tight')
            plt.show()
        
        # Conclusions
        print("\n" + "=" * 70)
        print("📋 CONCLUSIONS (AEMET Data)")
        print("=" * 70)
        
        for var in available_vars:
            label = var_labels.get(var, var)
            var_results = results_df[results_df['var_code'] == var]
            
            if len(var_results) == 0:
                continue
                
            avg_ratio = var_results['ratio'].mean()
            avg_stations = var_results['n_stations'].mean()
            
            print(f"\n{label}:")
            print(f"  Average stations with data: {avg_stations:.0f}")
            print(f"  Average Temporal/Spatial ratio: {avg_ratio:.1f}x")
            
            if avg_ratio > 5:
                print("  ✅ HIGHLY STABLE SPATIALLY → 1 station per island sufficient")
            elif avg_ratio > 2:
                print("  ⚠️ MODERATELY STABLE → 2-3 stations per island recommended")
            else:
                print("  ❌ SIGNIFICANT SPATIAL VARIATION → Multiple stations needed (4+)")
        
        # Station coverage summary
        print("\n" + "-" * 70)
        print("STATION COVERAGE:")
        for var in available_vars:
            label = var_labels.get(var, var)
            stations_with_var = df_bal[df_bal[var].notna()]['idema'].nunique()
            total_stations = df_bal['idema'].nunique()
            print(f"  {label}: {stations_with_var}/{total_stations} stations have data")

In [ ]:
print('STATIONS BY AVAILABILITY')
print('=' * 70)
for _, r in stations.sort_values('overall_pct', ascending=False).iterrows():
    print(f'{r["idema"]:<8} {r["name"]:<40} {r["alt"]:>5.0f}m  {r["overall_pct"]:>5.1f}%')

In [ ]:
# Compare each high altitude station vs island average
HIGH_ALT_THRESHOLD = 200  # meters

numeric_vars = ['ta', 'tamax', 'tamin', 'hr', 'pres', 'vv', 'vmax', 'prec']
numeric_vars = [v for v in numeric_vars if v in df_bal.columns]
high_stations_df = stations[stations['alt'] >= HIGH_ALT_THRESHOLD].copy()
island_avg = df_bal[numeric_vars].mean()


print('HIGH ALTITUDE STATIONS vs ISLAND AVERAGE')
print('=' * 120)

for _, station in high_stations_df.sort_values('alt', ascending=False).iterrows():
    station_data = df_bal[df_bal['idema'] == station['idema']]
    station_avg = station_data[numeric_vars].mean()
    
    print(f'\n{station["name"]} ({station["idema"]}) - {station["alt"]:.0f}m')
    print('-' * 80)
    print(f'{"Variable":<12} {"Station":>12} {"Island Avg":>12} {"Diff":>12} {"% Diff":>10}')
    print('-' * 80)
    
    for var in numeric_vars:
        st_val = station_avg[var]
        isl_val = island_avg[var]
        diff = st_val - isl_val
        pct = (diff / isl_val * 100) if isl_val != 0 else 0
        print(f'{var:<12} {st_val:>12.2f} {isl_val:>12.2f} {diff:>+12.2f} {pct:>+9.1f}%')

In [ ]:
print('=' * 70)
print('FINAL SUMMARY')
print('=' * 70)
print(f'''
AVAILABLE VARIABLES: {len(available_vars)}
{available_vars}

EXCLUDED (no data): {len(EXCLUDE_VARS)}
{EXCLUDE_VARS}

HIGHLY CORRELATED PAIRS: {len(high_corr_pairs)}
(Not removed - for your consideration when building models)

STATIONS: {len(stations)}
Altitude range: {stations["alt"].min():.0f}m - {stations["alt"].max():.0f}m

Best stations (>45% overall availability):
''')
for _, r in stations[stations['overall_pct'] >= 45].sort_values('overall_pct', ascending=False).iterrows():
    print(f'  {r["idema"]}: {r["name"]} ({r["alt"]:.0f}m) - {r["overall_pct"]:.1f}%')

In [ ]:
# Map of stations with low availability (<45%)
low_avail = stations[stations['overall_pct'] < 45].copy()

m_low = folium.Map(location=[stations['lat'].mean(), stations['lon'].mean()], zoom_start=8, tiles=None)

folium.TileLayer('https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
                 attr='Esri', name='Topographic').add_to(m_low)

for _, row in low_avail.iterrows():
    pct = row['overall_pct']
    color = '#f39c12' if pct >= 30 else '#e74c3c'
    popup = f'<b>{row["name"]}</b><br>ID: {row["idema"]}<br>Alt: {row["alt"]:.0f}m<br>Avail: {pct:.1f}%'
    folium.CircleMarker([row['lat'], row['lon']], radius=10, color='black', weight=2,
                        fill=True, fillColor=color, fillOpacity=0.85,
                        popup=folium.Popup(popup, max_width=250)).add_to(m_low)

folium.LayerControl().add_to(m_low)
m_low.save(str(OUTPUT_DIR / 'map_low_availability.html'))
print(f'Stations with <50% availability: {len(low_avail)}')
m_low

In [ ]:
# Export
avail_df.to_csv(OUTPUT_DIR / 'variable_availability.csv', index=False)
stations.to_csv(OUTPUT_DIR / 'station_availability.csv', index=False)
if len(high_corr_df) > 0:
    high_corr_df.to_csv(OUTPUT_DIR / 'high_correlations.csv', index=False)

print('Exported to results/')

In [ ]:
# Filter to remove stations above 300 meters

ALT_THRESHOLD = 70
coastal_stations = stations[stations['alt'] < ALT_THRESHOLD].copy()
stations = coastal_stations

In [ ]:
# Heatmap
heatmap_df = coastal_stations.sort_values('overall_pct', ascending=False).copy()
heatmap_df['label'] = heatmap_df['name'].str[:22] + ' (' + heatmap_df['alt'].astype(int).astype(str) + 'm)'
heatmap_data = heatmap_df.set_index('label')[pct_cols]
heatmap_data.columns = [AEMET_VARS[c.replace('_pct', '')] for c in heatmap_data.columns]

fig, ax = plt.subplots(figsize=(14, max(10, len(heatmap_data) * 0.35)))
sns.heatmap(heatmap_data, annot=True, fmt='.0f', cmap='RdYlGn', vmin=0, vmax=100, ax=ax,
            cbar_kws={'label': 'Availability %'}, annot_kws={'size': 8})
ax.set_title('Availability by Station and Variable (%)', fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'station_variable_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Get coastal station IDs
coastal_ids = stations['idema'].tolist()

# Filter df_bal to only coastal stations
df_coastal = df_bal[df_bal['idema'].isin(coastal_ids)].copy()

# Get available variables
available_vars = [v for v in ['ta', 'tamax', 'tamin', 'hr', 'prec', 'vv', 'vmax', 'dv', 'dmax', 'pres', 'inso'] 
                  if v in df_coastal.columns]

print(f'\nVariables for correlation: {available_vars}')

# Correlation matrix - ALL stations
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Left: All stations
corr_all = df_bal[available_vars].corr()
mask_all = np.triu(np.ones_like(corr_all, dtype=bool), k=1)
sns.heatmap(corr_all, mask=mask_all, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, vmin=-1, vmax=1, square=True, ax=axes[0], annot_kws={'size': 8})
axes[0].set_title(f'All Stations (n={df_bal["idema"].nunique()})', fontweight='bold', fontsize=12)

# Right: Coastal stations only
corr_coastal = df_coastal[available_vars].corr()
mask_coastal = np.triu(np.ones_like(corr_coastal, dtype=bool), k=1)
sns.heatmap(corr_coastal, mask=mask_coastal, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, vmin=-1, vmax=1, square=True, ax=axes[1], annot_kws={'size': 8})
axes[1].set_title(f'Coastal Stations Only (alt < {ALT_THRESHOLD}m, n={len(coastal_ids)})', fontweight='bold', fontsize=12)

plt.suptitle('Variable Correlation: All Stations vs Coastal Stations', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'correlation_matrix_comparison.png', dpi=150)
plt.show()

# Show correlation differences
print('\nCORRELATION DIFFERENCES (Coastal - All)')
print('=' * 60)
corr_diff = corr_coastal - corr_all
print('Significant changes (|diff| > 0.05):')
for i in range(len(available_vars)):
    for j in range(i+1, len(available_vars)):
        diff = corr_diff.iloc[i, j]
        if abs(diff) > 0.05:
            print(f'  {available_vars[i]:<6} - {available_vars[j]:<6}: {diff:+.3f}')

In [ ]:
from scipy.interpolate import LinearNDInterpolator
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import date

ISLANDS = {
    'Mallorca': {'lon_min': 2.3, 'lon_max': 3.5, 'lat_min': 39.2, 'lat_max': 39.95},
    'Menorca': {'lon_min': 3.8, 'lon_max': 4.35, 'lat_min': 39.8, 'lat_max': 40.1},
    'Ibiza': {'lon_min': 1.15, 'lon_max': 1.65, 'lat_min': 38.85, 'lat_max': 39.15},
}

interp_vars = ['prec', 'tamax', 'ta', 'tamin', 'hr', 'vv', 'dv', 'vmax', 'dmax']
interp_vars = [v for v in interp_vars if v in df_bal.columns]

VAR_LABELS = {
    'prec': 'Precipitation (mm)', 'tamax': 'Temp Max (°C)', 'ta': 'Temperature (°C)', 
    'tamin': 'Temp Min (°C)', 'hr': 'Humidity (%)', 'vv': 'Wind Speed (m/s)',
    'dv': 'Wind Dir (°)', 'vmax': 'Wind Gust (m/s)', 'dmax': 'Gust Dir (°)'
}

VAR_SCALES = {
    'prec': {'vmin': 0, 'vmax': 20, 'cmap': 'Blues'},
    'tamax': {'vmin': 0, 'vmax': 40, 'cmap': 'RdYlBu_r'},
    'ta': {'vmin': 0, 'vmax': 40, 'cmap': 'RdYlBu_r'},
    'tamin': {'vmin': 0, 'vmax': 40, 'cmap': 'RdYlBu_r'},
    'hr': {'vmin': 30, 'vmax': 100, 'cmap': 'BuGn'},
    'vv': {'vmin': 0, 'vmax': 15, 'cmap': 'YlOrRd'},
    'dv': {'vmin': 0, 'vmax': 360, 'cmap': 'hsv'},
    'vmax': {'vmin': 0, 'vmax': 25, 'cmap': 'YlOrRd'},
    'dmax': {'vmin': 0, 'vmax': 360, 'cmap': 'hsv'},
}

def detect_island(row):
    lat, lon = row['lat'], row['lon']
    for island, bbox in ISLANDS.items():
        if bbox['lon_min'] <= lon <= bbox['lon_max'] and bbox['lat_min'] <= lat <= bbox['lat_max']:
            return island
    return 'Unknown'

coastal_stations['island'] = coastal_stations.apply(detect_island, axis=1)
dates = sorted(df_bal['fint'].dt.date.unique())

date_dropdown = widgets.Dropdown(options=dates, description='Date:', value=date(2022, 10, 26))
var_dropdown = widgets.Dropdown(options=interp_vars, description='Variable:', value='ta')
island_dropdown = widgets.Dropdown(options=list(ISLANDS.keys()), description='Island:', value='Mallorca')
btn = widgets.Button(description='Show Interpolation')
output = widgets.Output()

def plot_interpolation(b):
    with output:
        clear_output(wait=True)
        
        sel_date = date_dropdown.value
        sel_var = var_dropdown.value
        sel_island = island_dropdown.value
        bbox = ISLANDS[sel_island]
        scale = VAR_SCALES[sel_var]
        
        day_data = df_bal[df_bal['fint'].dt.date == sel_date]
        island_stations = coastal_stations[coastal_stations['island'] == sel_island]
        day_island = day_data[day_data['idema'].isin(island_stations['idema'])]
        
        station_vals = day_island.groupby('idema')[sel_var].mean().reset_index()
        station_vals = station_vals.merge(island_stations[['idema', 'lon', 'lat', 'name']], on='idema')
        station_vals = station_vals.dropna(subset=[sel_var])
        
        if len(station_vals) < 3:
            print(f'Not enough stations ({len(station_vals)})')
            return
        
        points = station_vals[['lon', 'lat']].values
        values = station_vals[sel_var].values
        
        grid_lon = np.linspace(bbox['lon_min'], bbox['lon_max'], 50)
        grid_lat = np.linspace(bbox['lat_min'], bbox['lat_max'], 50)
        grid_lon_2d, grid_lat_2d = np.meshgrid(grid_lon, grid_lat)
        
        linear_interp = LinearNDInterpolator(points, values)
        grid_values = linear_interp(grid_lon_2d, grid_lat_2d)
        
        levels = np.linspace(scale['vmin'], scale['vmax'], 40)
        
        fig, ax = plt.subplots(figsize=(10, 8))
        im = ax.contourf(grid_lon_2d, grid_lat_2d, grid_values, levels=levels,
                        cmap=scale['cmap'], vmin=scale['vmin'], vmax=scale['vmax'], 
                        alpha=0.8, extend='both')
        plt.colorbar(im, ax=ax, label=VAR_LABELS[sel_var])
        ax.scatter(points[:, 0], points[:, 1], c='black', s=60, zorder=5, 
                  edgecolors='white', linewidth=1.5)
        for _, row in station_vals.iterrows():
            ax.annotate(f'{row[sel_var]:.1f}', (row['lon'], row['lat']), fontsize=8,
                       xytext=(3, 3), textcoords='offset points',
                       bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
        ax.set_xlim(bbox['lon_min'], bbox['lon_max'])
        ax.set_ylim(bbox['lat_min'], bbox['lat_max'])
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
        ax.set_title(f'{sel_island} - {VAR_LABELS[sel_var]} - {sel_date}', fontweight='bold', fontsize=12)
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        coverage = (~np.isnan(grid_values)).mean() * 100
        print(f'\nLinear Interpolation Stats:')
        print(f'  Coverage: {coverage:.1f}%')
        print(f'  Mean: {np.nanmean(grid_values):.2f}')
        print(f'  Min: {np.nanmin(grid_values):.2f}')
        print(f'  Max: {np.nanmax(grid_values):.2f}')

btn.on_click(plot_interpolation)
display(widgets.HBox([date_dropdown, var_dropdown, island_dropdown, btn]))
display(output)

In [ ]:
from scipy.interpolate import LinearNDInterpolator, NearestNDInterpolator
import numpy as np
import pandas as pd

def get_weather_at_location(df_bal, stations, target_date, target_lat, target_lon, variables=None):
    """
    Get interpolated weather data for a specific date and location.
    
    Parameters:
        df_bal: DataFrame with weather observations
        stations: DataFrame with station metadata (idema, lat, lon)
        target_date: date object or string 'YYYY-MM-DD'
        target_lat: latitude of target location
        target_lon: longitude of target location
        variables: list of variables to interpolate (default: all available)
    
    Returns:
        dict with interpolated values for each variable
    """
    if isinstance(target_date, str):
        target_date = pd.to_datetime(target_date).date()
    
    if variables is None:
        variables = ['ta', 'tamax', 'tamin', 'hr', 'prec', 'vv', 'vmax', 'dv', 'pres']
    variables = [v for v in variables if v in df_bal.columns]
    
    day_data = df_bal[df_bal['fint'].dt.date == target_date]
    
    if len(day_data) == 0:
        return {'error': f'No data for {target_date}'}
    
    results = {'date': target_date, 'lat': target_lat, 'lon': target_lon}
    
    for var in variables:
        station_vals = day_data.groupby('idema')[var].mean().reset_index()
        station_vals = station_vals.merge(stations[['idema', 'lon', 'lat']], on='idema')
        station_vals = station_vals.dropna(subset=[var])
        
        if len(station_vals) < 3:
            results[var] = None
            continue
        
        points = station_vals[['lon', 'lat']].values
        values = station_vals[var].values
        
        linear_interp = LinearNDInterpolator(points, values)
        nearest_interp = NearestNDInterpolator(points, values)
        
        value = linear_interp(target_lon, target_lat)
        if np.isnan(value):
            value = nearest_interp(target_lon, target_lat)
            results[f'{var}_method'] = 'nearest'
        else:
            results[f'{var}_method'] = 'linear'
        
        results[var] = float(value)
    
    return results


# Example usage
target_date = '2022-10-26'
target_lat = 39.5695  
target_lon = 2.7396

result = get_weather_at_location(df_bal, stations, target_date, target_lat, target_lon)

print(f"Weather at ({target_lat}, {target_lon}) on {target_date}:")
print("=" * 50)
for key, val in result.items():
    if key in ['date', 'lat', 'lon'] or '_method' in key:
        continue
    method = result.get(f'{key}_method', '')
    if val is not None:
        print(f"  {key:>8}: {val:>8.2f}  ({method})")
    else:
        print(f"  {key:>8}: No data")

In [ ]:
# Highest STD analysis - by variable, day, and station
print('SPATIAL VARIABILITY ANALYSIS (Standard Deviation)')
print('=' * 80)

df_bal['date'] = df_bal['fint'].dt.date
analysis_vars = ['prec', 'tamax', 'ta', 'tamin', 'hr', 'vv', 'vmax']
analysis_vars = [v for v in analysis_vars if v in df_bal.columns]

# 1. Overall STD per variable
print('\n1. OVERALL STD BY VARIABLE')
print('-' * 40)
for var in analysis_vars:
    std_val = df_bal[var].std()
    print(f'{var:<10} {VAR_LABELS.get(var, var):<25} STD: {std_val:>8.2f}')

# 2. Day with highest spatial STD per variable
print('\n2. DAY WITH HIGHEST SPATIAL VARIABILITY (per variable)')
print('-' * 80)
print(f'{"Variable":<10} {"Date":<12} {"STD":>8} {"Mean":>8} {"Min":>8} {"Max":>8}')
print('-' * 80)

high_std_days = {}
for var in analysis_vars:
    daily_std = df_bal.groupby('date')[var].std()
    max_std_date = daily_std.idxmax()
    max_std = daily_std.max()
    day_data = df_bal[df_bal['date'] == max_std_date][var]
    high_std_days[var] = {'date': max_std_date, 'std': max_std}
    print(f'{var:<10} {str(max_std_date):<12} {max_std:>8.2f} {day_data.mean():>8.2f} {day_data.min():>8.2f} {day_data.max():>8.2f}')

# 3. Station with highest STD per variable
print('\n3. STATION WITH HIGHEST TEMPORAL VARIABILITY (per variable)')
print('-' * 80)
print(f'{"Variable":<10} {"Station":<30} {"STD":>8} {"Mean":>8}')
print('-' * 80)

for var in analysis_vars:
    station_std = df_bal.groupby('idema')[var].std().dropna()
    if len(station_std) == 0:
        print(f'{var:<10} {"No data":<30}')
        continue
    max_std_station = station_std.idxmax()
    max_std = station_std.max()
    station_match = stations[stations['idema'] == max_std_station]
    if len(station_match) > 0:
        station_name = station_match['name'].values[0]
    else:
        station_name = max_std_station
    station_mean = df_bal[df_bal['idema'] == max_std_station][var].mean()
    print(f'{var:<10} {str(station_name)[:28]:<30} {max_std:>8.2f} {station_mean:>8.2f}')

# 4. Heatmap: Daily STD per variable
print('\n4. DAILY SPATIAL STD HEATMAP')
daily_std_df = df_bal.groupby('date')[analysis_vars].std()

fig, ax = plt.subplots(figsize=(14, 8))
sns.heatmap(daily_std_df.T, cmap='YlOrRd', ax=ax, cbar_kws={'label': 'STD'})
ax.set_xlabel('Date')
ax.set_ylabel('Variable')
ax.set_title('Daily Spatial Standard Deviation by Variable', fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'daily_std_heatmap.png', dpi=150)
plt.show()

# 5. Top 10 days with highest overall variability
print('\n5. TOP 10 DAYS WITH HIGHEST OVERALL VARIABILITY')
print('-' * 60)
daily_std_df['total_std'] = daily_std_df[['ta', 'hr', 'vv']].mean(axis=1)
top_days = daily_std_df.nlargest(10, 'total_std')
print(f'{"Date":<15} {"ta STD":>10} {"hr STD":>10} {"vv STD":>10}')
print('-' * 60)
for date, row in top_days.iterrows():
    print(f'{str(date):<15} {row["ta"]:>10.2f} {row["hr"]:>10.2f} {row["vv"]:>10.2f}')

In [ ]:
# from scipy.spatial import Delaunay, ConvexHull
# from scipy.interpolate import LinearNDInterpolator, NearestNDInterpolator, RBFInterpolator
# from scipy.spatial.distance import cdist
# from sklearn.neighbors import KNeighborsRegressor
# import numpy as np
# import matplotlib.pyplot as plt
# import ipywidgets as widgets
# from IPython.display import display, clear_output

# class HullMultipointInterpolator:
#     """Linear inside hull + IDW on boundary samples outside."""
    
#     def __init__(self, n_boundary_samples=50, k_neighbors=10, power=2):
#         self.n_boundary_samples = n_boundary_samples
#         self.k_neighbors = k_neighbors
#         self.power = power
        
#     def _sample_hull_boundary(self, points, n_samples):
#         """Sample points along hull boundary using scipy ConvexHull."""
#         hull = ConvexHull(points)
#         hull_vertices = points[hull.vertices]
#         hull_closed = np.vstack([hull_vertices, hull_vertices[0]])
        
#         edges = np.diff(hull_closed, axis=0)
#         edge_lengths = np.linalg.norm(edges, axis=1)
#         total_length = edge_lengths.sum()
        
#         sample_distances = np.linspace(0, total_length, n_samples, endpoint=False)
        
#         boundary_pts = []
#         cumulative = 0
#         edge_idx = 0
        
#         for d in sample_distances:
#             while edge_idx < len(edge_lengths) - 1 and cumulative + edge_lengths[edge_idx] < d:
#                 cumulative += edge_lengths[edge_idx]
#                 edge_idx += 1
            
#             t = (d - cumulative) / edge_lengths[edge_idx] if edge_lengths[edge_idx] > 0 else 0
#             t = np.clip(t, 0, 1)
#             pt = hull_closed[edge_idx] + t * edges[edge_idx]
#             boundary_pts.append(pt)
        
#         return np.array(boundary_pts), hull_closed
        
#     def fit(self, points, values):
#         self.points = np.asarray(points)
#         self.values = np.asarray(values)
        
#         self.linear = LinearNDInterpolator(self.points, self.values)
#         self.delaunay = Delaunay(self.points)
#         self.boundary_pts, self.hull_closed = self._sample_hull_boundary(self.points, self.n_boundary_samples)
#         self.boundary_vals = self.linear(self.boundary_pts)
        
#         valid = ~np.isnan(self.boundary_vals)
#         self.boundary_pts = self.boundary_pts[valid]
#         self.boundary_vals = self.boundary_vals[valid]
        
#         self.knn = KNeighborsRegressor(
#             n_neighbors=min(self.k_neighbors, len(self.boundary_pts)),
#             weights='distance', p=self.power
#         )
#         self.knn.fit(self.boundary_pts, self.boundary_vals)
#         return self
    
#     def predict(self, targets):
#         targets = np.asarray(targets)
#         if targets.ndim == 1:
#             targets = targets.reshape(1, -1)
        
#         result = np.zeros(len(targets))
#         inside_mask = self.delaunay.find_simplex(targets) >= 0
        
#         if inside_mask.any():
#             result[inside_mask] = self.linear(targets[inside_mask])
#         if (~inside_mask).any():
#             result[~inside_mask] = self.knn.predict(targets[~inside_mask])
        
#         return result

# # ============================================================
# # INTERPOLATION METHODS
# # ============================================================
# def find_nearest_point_on_hull(hull_points, target_point):
#     """Find nearest point on convex hull boundary."""
#     hull = ConvexHull(hull_points)
#     min_dist = np.inf
#     nearest_point = None
    
#     for simplex in hull.simplices:
#         p1, p2 = hull_points[simplex[0]], hull_points[simplex[1]]
#         edge_vec = p2 - p1
#         edge_len_sq = np.dot(edge_vec, edge_vec)
        
#         if edge_len_sq == 0:
#             nearest_on_edge = p1
#         else:
#             t = max(0, min(1, np.dot(target_point - p1, edge_vec) / edge_len_sq))
#             nearest_on_edge = p1 + t * edge_vec
        
#         dist = np.linalg.norm(target_point - nearest_on_edge)
#         if dist < min_dist:
#             min_dist = dist
#             nearest_point = nearest_on_edge
    
#     return nearest_point, min_dist

# def interpolate_with_method(points, values, grid_pts, method):
#     """Apply selected interpolation method."""
    
#     if method == 'linear':
#         interp = LinearNDInterpolator(points, values)
#         return interp(grid_pts), 'Linear (NaN outside hull)'
    
#     elif method == 'nearest':
#         interp = NearestNDInterpolator(points, values)
#         return interp(grid_pts), 'Nearest Neighbor'
    
#     elif method == 'linear_nearest':
#         linear = LinearNDInterpolator(points, values)
#         nearest = NearestNDInterpolator(points, values)
#         linear_vals = linear(grid_pts)
#         nearest_vals = nearest(grid_pts)
#         return np.where(np.isnan(linear_vals), nearest_vals, linear_vals), 'Linear + Nearest Station'
    
#     elif method == 'linear_hull':
#         linear = LinearNDInterpolator(points, values)
#         linear_vals = linear(grid_pts)
#         try:
#             hull_check = Delaunay(points)
#         except:
#             nearest = NearestNDInterpolator(points, values)
#             return np.where(np.isnan(linear_vals), nearest(grid_pts), linear_vals), 'Linear + Nearest'
        
#         result = linear_vals.copy()
#         for i, (pt, val) in enumerate(zip(grid_pts, linear_vals)):
#             if np.isnan(val):
#                 nearest_on_hull, _ = find_nearest_point_on_hull(points, pt)
#                 result[i] = linear(nearest_on_hull.reshape(1, -1))[0]
#                 if np.isnan(result[i]):
#                     nearest = NearestNDInterpolator(points, values)
#                     result[i] = nearest(pt.reshape(1, -1))[0]
#         return result, 'Linear + Hull Boundary (basic)'
    
#     elif method == 'hull_gradient':
#         linear = LinearNDInterpolator(points, values)
#         try:
#             hull_check = Delaunay(points)
#         except:
#             nearest = NearestNDInterpolator(points, values)
#             return nearest(grid_pts), 'Nearest (hull failed)'
        
#         result = linear(grid_pts)
#         epsilon = 0.01
        
#         for i, (pt, val) in enumerate(zip(grid_pts, result)):
#             if np.isnan(val):
#                 nearest_on_hull, dist = find_nearest_point_on_hull(points, pt)
#                 hull_val = linear(nearest_on_hull.reshape(1, -1))[0]
                
#                 if np.isnan(hull_val):
#                     nearest = NearestNDInterpolator(points, values)
#                     result[i] = nearest(pt.reshape(1, -1))[0]
#                     continue
                
#                 direction = pt - nearest_on_hull
#                 dir_norm = np.linalg.norm(direction)
                
#                 if dir_norm > 0:
#                     direction = direction / dir_norm
#                     inside_pt = nearest_on_hull - epsilon * direction
#                     inside_val = linear(inside_pt.reshape(1, -1))[0]
                    
#                     if not np.isnan(inside_val):
#                         gradient = (hull_val - inside_val) / epsilon
#                         decay = np.exp(-dist * 2)
#                         result[i] = hull_val + gradient * dist * decay
#                     else:
#                         result[i] = hull_val
#                 else:
#                     result[i] = hull_val
        
#         return result, 'Linear + Hull Gradient'
    
#     elif method == 'hull_decay':
#         linear = LinearNDInterpolator(points, values)
#         try:
#             hull_check = Delaunay(points)
#         except:
#             nearest = NearestNDInterpolator(points, values)
#             return nearest(grid_pts), 'Nearest (hull failed)'
        
#         result = linear(grid_pts)
#         regional_mean = np.mean(values)
#         decay_rate = 3.0
        
#         for i, (pt, val) in enumerate(zip(grid_pts, result)):
#             if np.isnan(val):
#                 nearest_on_hull, dist = find_nearest_point_on_hull(points, pt)
#                 hull_val = linear(nearest_on_hull.reshape(1, -1))[0]
                
#                 if np.isnan(hull_val):
#                     nearest = NearestNDInterpolator(points, values)
#                     hull_val = nearest(pt.reshape(1, -1))[0]
                
#                 weight = np.exp(-decay_rate * dist)
#                 result[i] = weight * hull_val + (1 - weight) * regional_mean
        
#         return result, 'Linear + Hull Decay to Mean'
    
#     elif method == 'hull_idw_blend':
#         linear = LinearNDInterpolator(points, values)
#         try:
#             hull_check = Delaunay(points)
#         except:
#             return idw_interpolate(points, values, grid_pts, 2), 'IDW (hull failed)'
        
#         result = linear(grid_pts)
        
#         distances = cdist(grid_pts, points)
#         distances = np.maximum(distances, 1e-10)
#         weights = 1 / distances**2
#         weights_norm = weights / weights.sum(axis=1, keepdims=True)
#         idw_vals = (weights_norm * values).sum(axis=1)
        
#         blend_distance = 0.1
        
#         for i, (pt, val) in enumerate(zip(grid_pts, result)):
#             if np.isnan(val):
#                 nearest_on_hull, dist = find_nearest_point_on_hull(points, pt)
#                 hull_val = linear(nearest_on_hull.reshape(1, -1))[0]
                
#                 if np.isnan(hull_val):
#                     result[i] = idw_vals[i]
#                     continue
                
#                 blend = 1 - np.exp(-dist / blend_distance)
#                 result[i] = (1 - blend) * hull_val + blend * idw_vals[i]
        
#         return result, 'Linear + Hull-IDW Blend'
    
#     elif method == 'hull_multipoint':
#         # Use the class-based implementation
#         try:
#             interp = HullMultipointInterpolator(n_boundary_samples=50, k_neighbors=10)
#             interp.fit(points, values)
#             return interp.predict(grid_pts), 'Linear + Hull Multipoint (sklearn)'
#         except:
#             distances = cdist(grid_pts, points)
#             distances = np.maximum(distances, 1e-10)
#             weights = 1 / distances**2
#             weights_norm = weights / weights.sum(axis=1, keepdims=True)
#             return (weights_norm * values).sum(axis=1), 'IDW (hull_multipoint failed)'
    
#     elif method == 'idw2':
#         distances = cdist(grid_pts, points)
#         distances = np.maximum(distances, 1e-10)
#         weights = 1 / distances**2
#         weights_norm = weights / weights.sum(axis=1, keepdims=True)
#         return (weights_norm * values).sum(axis=1), 'IDW (power=2)'
    
#     elif method == 'idw3':
#         distances = cdist(grid_pts, points)
#         distances = np.maximum(distances, 1e-10)
#         weights = 1 / distances**3
#         weights_norm = weights / weights.sum(axis=1, keepdims=True)
#         return (weights_norm * values).sum(axis=1), 'IDW (power=3)'
    
#     elif method == 'rbf_smooth':
#         try:
#             rbf = RBFInterpolator(points, values, kernel='thin_plate_spline', smoothing=0.5)
#             result = rbf(grid_pts)
#             val_min, val_max = values.min(), values.max()
#             val_range = val_max - val_min
#             result = np.clip(result, val_min - 0.2 * val_range, val_max + 0.2 * val_range)
#             return result, 'RBF Smooth (clipped)'
#         except:
#             return interpolate_with_method(points, values, grid_pts, 'idw2')
    
#     elif method == 'rbf_tps':
#         try:
#             rbf = RBFInterpolator(points, values, kernel='thin_plate_spline')
#             return rbf(grid_pts), 'RBF (thin plate spline)'
#         except:
#             return interpolate_with_method(points, values, grid_pts, 'idw2')
    
#     elif method == 'knn5':
#         k = min(5, len(points))
#         knn = KNeighborsRegressor(n_neighbors=k, weights='distance')
#         knn.fit(points, values)
#         return knn.predict(grid_pts), 'KNN (k=5, sklearn)'
    
#     return LinearNDInterpolator(points, values)(grid_pts), 'Linear'

# # ============================================================
# # WIDGET INTERFACE
# # ============================================================
# available_dates = sorted(df_bal['fint'].dt.date.unique())
# available_vars = [v for v in AEMET_VARS.keys() if v in df_bal.columns]
# island_options = ['All'] + list(ISLANDS.keys())

# method_options = [
#     ('Linear (NaN outside)', 'linear'),
#     ('Nearest Neighbor', 'nearest'),
#     ('Linear + Nearest Station', 'linear_nearest'),
#     ('Linear + Hull Boundary (basic)', 'linear_hull'),
#     ('─── IMPROVED HULL METHODS ───', 'separator'),
#     ('Linear + Hull Gradient', 'hull_gradient'),
#     ('Linear + Hull Decay to Mean', 'hull_decay'),
#     ('Linear + Hull-IDW Blend', 'hull_idw_blend'),
#     ('Linear + Hull Multipoint (sklearn)', 'hull_multipoint'),
#     ('─── OTHER METHODS ───', 'separator2'),
#     ('IDW (power=2)', 'idw2'),
#     ('IDW (power=3)', 'idw3'),
#     ('RBF Smooth (clipped)', 'rbf_smooth'),
#     ('RBF (thin plate spline)', 'rbf_tps'),
#     ('KNN (k=5, sklearn)', 'knn5'),
# ]

# from datetime import date
# date_dropdown = widgets.Dropdown(options=available_dates, value=date(2022, 10, 26), description='Date:')
# var_dropdown = widgets.Dropdown(options=[(AEMET_VARS[v], v) for v in available_vars], value='hr', description='Variable:')
# island_dropdown = widgets.Dropdown(options=island_options, value='Mallorca', description='Island:')
# method_dropdown = widgets.Dropdown(options=method_options, value='hull_multipoint', description='Method:')
# show_hull = widgets.Checkbox(value=True, description='Show Hull')
# show_labels = widgets.Checkbox(value=True, description='Show Labels')
# show_boundary = widgets.Checkbox(value=False, description='Show Boundary Samples')
# show_btn = widgets.Button(description='Show Interpolation', button_style='primary')
# compare_btn = widgets.Button(description='Compare Hull Methods', button_style='info')
# output = widgets.Output()

# def get_station_data(sel_date, sel_var, sel_island):
#     day_data = df_bal[df_bal['fint'].dt.date == sel_date]
#     station_data = day_data.groupby(['idema', 'lat', 'lon']).agg({sel_var: 'mean'}).reset_index().dropna()
    
#     if sel_island and sel_island != 'All':
#         bbox = ISLANDS[sel_island]
#         station_data = station_data[
#             (station_data['lon'] >= bbox['lon_min'] - 0.2) & 
#             (station_data['lon'] <= bbox['lon_max'] + 0.2) &
#             (station_data['lat'] >= bbox['lat_min'] - 0.2) & 
#             (station_data['lat'] <= bbox['lat_max'] + 0.2)
#         ]
#     return station_data

# def show_interpolation(b):
#     with output:
#         clear_output(wait=True)
        
#         sel_date = date_dropdown.value
#         sel_var = var_dropdown.value
#         sel_island = island_dropdown.value if island_dropdown.value != 'All' else None
#         sel_method = method_dropdown.value
        
#         if 'separator' in sel_method:
#             print("Please select an actual method")
#             return
        
#         station_data = get_station_data(sel_date, sel_var, sel_island)
        
#         if len(station_data) < 2:
#             print(f"Not enough stations")
#             return
        
#         points = station_data[['lon', 'lat']].values
#         values = station_data[sel_var].values
        
#         bbox = ISLANDS[sel_island] if sel_island else BBOX
#         grid_lon = np.linspace(bbox['lon_min'], bbox['lon_max'], 60)
#         grid_lat = np.linspace(bbox['lat_min'], bbox['lat_max'], 60)
#         grid_lon_2d, grid_lat_2d = np.meshgrid(grid_lon, grid_lat)
#         grid_pts = np.column_stack([grid_lon_2d.ravel(), grid_lat_2d.ravel()])
        
#         grid_values, method_label = interpolate_with_method(points, values, grid_pts, sel_method)
#         grid_values = grid_values.reshape(grid_lon_2d.shape)
        
#         fig, ax = plt.subplots(figsize=(10, 8))
#         im = ax.contourf(grid_lon_2d, grid_lat_2d, grid_values, levels=20, cmap='RdYlBu_r', extend='both')
#         ax.scatter(points[:, 0], points[:, 1], c='black', s=80, marker='^', edgecolors='white', linewidth=1.5, zorder=5)
        
#         if show_hull.value:
#             try:
#                 hull = ConvexHull(points)
#                 hull_pts = points[hull.vertices]
#                 hull_closed = np.vstack([hull_pts, hull_pts[0]])
#                 ax.plot(hull_closed[:, 0], hull_closed[:, 1], 'k--', linewidth=2, alpha=0.7)
#             except:
#                 pass
        
#         # Show boundary samples for hull_multipoint
#         if show_boundary.value and sel_method == 'hull_multipoint':
#             try:
#                 interp = HullMultipointInterpolator(n_boundary_samples=50, k_neighbors=10)
#                 interp.fit(points, values)
#                 ax.scatter(interp.boundary_pts[:, 0], interp.boundary_pts[:, 1], 
#                           c='lime', s=20, alpha=0.8, zorder=4, label='Boundary Samples')
#                 ax.legend(loc='upper right')
#             except:
#                 pass
        
#         if show_labels.value:
#             for _, row in station_data.iterrows():
#                 ax.annotate(f'{row[sel_var]:.1f}', (row['lon'], row['lat']), 
#                            fontsize=8, ha='center', va='bottom', xytext=(0, 5), 
#                            textcoords='offset points',
#                            bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
        
#         plt.colorbar(im, ax=ax, label=VAR_LABELS.get(sel_var, sel_var))
#         ax.set_xlabel('Longitude')
#         ax.set_ylabel('Latitude')
#         ax.set_title(f'{VAR_LABELS.get(sel_var, sel_var)} - {sel_date}\n{sel_island or "All"} | {method_label}', fontweight='bold')
#         ax.grid(True, alpha=0.3)
#         plt.tight_layout()
#         plt.show()
        
#         print(f"Stations: {len(station_data)} | Range: {values.min():.1f} - {values.max():.1f}")

# def compare_hull_methods(b):
#     with output:
#         clear_output(wait=True)
        
#         sel_date = date_dropdown.value
#         sel_var = var_dropdown.value
#         sel_island = island_dropdown.value if island_dropdown.value != 'All' else None
        
#         station_data = get_station_data(sel_date, sel_var, sel_island)
        
#         if len(station_data) < 2:
#             print(f"Not enough stations")
#             return
        
#         points = station_data[['lon', 'lat']].values
#         values = station_data[sel_var].values
        
#         bbox = ISLANDS[sel_island] if sel_island else BBOX
#         grid_lon = np.linspace(bbox['lon_min'], bbox['lon_max'], 50)
#         grid_lat = np.linspace(bbox['lat_min'], bbox['lat_max'], 50)
#         grid_lon_2d, grid_lat_2d = np.meshgrid(grid_lon, grid_lat)
#         grid_pts = np.column_stack([grid_lon_2d.ravel(), grid_lat_2d.ravel()])
        
#         methods = ['linear_nearest', 'linear_hull', 'hull_gradient', 'hull_decay', 'hull_idw_blend', 'hull_multipoint']
        
#         fig, axes = plt.subplots(2, 3, figsize=(15, 10))
#         axes = axes.flatten()
        
#         for idx, method in enumerate(methods):
#             ax = axes[idx]
#             grid_values, method_label = interpolate_with_method(points, values, grid_pts, method)
#             grid_values = grid_values.reshape(grid_lon_2d.shape)
            
#             im = ax.contourf(grid_lon_2d, grid_lat_2d, grid_values, levels=15, cmap='RdYlBu_r', extend='both')
#             ax.scatter(points[:, 0], points[:, 1], c='black', s=40, marker='^', edgecolors='white')
            
#             try:
#                 hull = ConvexHull(points)
#                 hull_pts = points[hull.vertices]
#                 hull_closed = np.vstack([hull_pts, hull_pts[0]])
#                 ax.plot(hull_closed[:, 0], hull_closed[:, 1], 'k--', linewidth=1.5, alpha=0.6)
#             except:
#                 pass
            
#             ax.set_title(method_label, fontweight='bold', fontsize=9)
#             plt.colorbar(im, ax=ax)
        
#         plt.suptitle(f'{VAR_LABELS.get(sel_var, sel_var)} - {sel_date} | Comparison of Interpolation Methods', 
#                      fontsize=12, fontweight='bold', y=1.02)
#         plt.tight_layout()
#         plt.savefig(OUTPUT_DIR / 'hull_methods_comparison.png', dpi=150, bbox_inches='tight')
#         plt.show()
        
#         print(f"Saved to {OUTPUT_DIR}/hull_methods_comparison.png")

# show_btn.on_click(show_interpolation)
# compare_btn.on_click(compare_hull_methods)

# print("INTERPOLATION EXPLORER (with Hull Multipoint using sklearn)")
# print("=" * 60)
# display(widgets.VBox([
#     widgets.HBox([date_dropdown, var_dropdown, island_dropdown]),
#     widgets.HBox([method_dropdown, show_hull, show_labels, show_boundary]),
#     widgets.HBox([show_btn, compare_btn])
# ]))
# display(output)

In [ ]:
import requests
import json
import hashlib
from pathlib import Path
from datetime import date, timedelta
import numpy as np
import pandas as pd
from scipy.spatial import Delaunay, ConvexHull
from scipy.interpolate import LinearNDInterpolator
from sklearn.neighbors import KNeighborsRegressor

# Cache
CACHE_DIR = Path('cache/openmeteo')
CACHE_DIR.mkdir(parents=True, exist_ok=True)

# Variables
AEMET_VARS = ['ta', 'hr', 'prec', 'vv', 'dv', 'pres', 'tamin', 'tamax', 'inso', 'pres_nmar', 'ts']
OPENMETEO_VARS = [
    'temperature_2m', 'apparent_temperature', 'dewpoint_2m', 'relative_humidity_2m',
    'pressure_msl', 'precipitation', 'rain', 'wind_speed_10m', 'wind_direction_10m',
    'wind_gusts_10m', 'cloud_cover', 'cloud_cover_low', 'cloud_cover_mid', 'cloud_cover_high',
    'visibility', 'uv_index', 'uv_index_clear_sky', 'sunshine_duration',
    'evapotranspiration', 'vapour_pressure_deficit'
]

def _get_openmeteo(lat, lon, target_date):
    """Fetch Open-Meteo data with cache."""
    if hasattr(target_date, 'date'):
        target_date = target_date.date()
    
    start = target_date - timedelta(days=1)
    end = target_date + timedelta(days=1)
    
    cache_key = hashlib.md5(f"{lat:.4f}_{lon:.4f}_{start}_{end}".encode()).hexdigest()
    cache_file = CACHE_DIR / f"{cache_key}.json"
    
    if cache_file.exists():
        with open(cache_file, 'r') as f:
            data = json.load(f)
    else:
        try:
            resp = requests.get(
                "https://archive-api.open-meteo.com/v1/archive",
                params={
                    'latitude': lat, 'longitude': lon,
                    'start_date': str(start), 'end_date': str(end),
                    'hourly': ','.join(OPENMETEO_VARS),
                    'timezone': 'Europe/Madrid'
                },
                timeout=30
            )
            resp.raise_for_status()
            data = resp.json()
            with open(cache_file, 'w') as f:
                json.dump(data, f)
        except:
            return {}
    
    if 'hourly' not in data:
        return {}
    
    times = pd.to_datetime(data['hourly']['time'])
    mask = times.date == target_date
    
    result = {}
    for var in OPENMETEO_VARS:
        if var in data['hourly']:
            vals = np.array(data['hourly'][var])[mask]
            vals = vals[~pd.isna(vals)]
            if len(vals) > 0:
                result[var] = float(np.mean(vals))
    return result

def _get_aemet(df, lat, lon, target_date, var):
    """Get interpolated AEMET value."""
    if hasattr(target_date, 'date'):
        target_date = target_date.date()
    
    if var not in df.columns:
        return None
    
    day_data = df[df['fint'].dt.date == target_date]
    stations = day_data.groupby(['idema', 'lat', 'lon']).agg({var: 'mean'}).reset_index()
    
    # Fill missing with spatial KNN
    missing = stations[var].isna()
    if missing.any() and (~missing).sum() >= 2:
        knn = KNeighborsRegressor(n_neighbors=min(3, (~missing).sum()), weights='distance')
        knn.fit(stations.loc[~missing, ['lon', 'lat']].values, stations.loc[~missing, var].values)
        stations.loc[missing, var] = knn.predict(stations.loc[missing, ['lon', 'lat']].values)
    
    stations = stations.dropna(subset=[var])
    if len(stations) < 3:
        return None
    
    points = stations[['lon', 'lat']].values
    values = stations[var].values
    target = np.array([[lon, lat]])
    
    linear = LinearNDInterpolator(points, values)
    delaunay = Delaunay(points)
    
    # Inside hull
    if delaunay.find_simplex(target)[0] >= 0:
        return float(linear(target)[0])
    
    # Outside hull: boundary KNN
    hull = ConvexHull(points)
    hull_closed = np.vstack([points[hull.vertices], points[hull.vertices[0]]])
    edges = np.diff(hull_closed, axis=0)
    lengths = np.linalg.norm(edges, axis=1)
    total = lengths.sum()
    
    boundary_pts = []
    cum, idx = 0, 0
    for d in np.linspace(0, total, 50, endpoint=False):
        while idx < len(lengths) - 1 and cum + lengths[idx] < d:
            cum += lengths[idx]
            idx += 1
        t = (d - cum) / lengths[idx] if lengths[idx] > 0 else 0
        boundary_pts.append(hull_closed[idx] + np.clip(t, 0, 1) * edges[idx])
    
    boundary_pts = np.array(boundary_pts)
    boundary_vals = linear(boundary_pts)
    valid = ~np.isnan(boundary_vals)
    boundary_pts, boundary_vals = boundary_pts[valid], boundary_vals[valid]
    
    if len(boundary_pts) < 2:
        return None
    
    knn = KNeighborsRegressor(n_neighbors=min(10, len(boundary_pts)), weights='distance')
    knn.fit(boundary_pts, boundary_vals)
    return float(knn.predict(target)[0])

def get_all_weather(df_aemet, lat, lon, target_date):
    """
    Get all weather variables from AEMET and Open-Meteo.
    
    Parameters:
        df_aemet: DataFrame with AEMET data
        lat, lon: coordinates
        target_date: date object
    
    Returns:
        dict with 'ae_*' (AEMET) and 'om_*' (OpenMeteo) keys
    """
    result = {}
    
    # AEMET
    for var in AEMET_VARS:
        val = _get_aemet(df_aemet, lat, lon, target_date, var)
        if val is not None:
            result[f'ae_{var}'] = val
    
    # OpenMeteo
    om = _get_openmeteo(lat, lon, target_date)
    for var, val in om.items():
        result[f'om_{var}'] = val
    
    return result

# ============================================================
# USAGE
# ============================================================
print("WEATHER MODULE LOADED")
print("=" * 40)
print("Usage: get_all_weather(df_aemet, lat, lon, date)")
print()

# Example
weather = get_all_weather(df_bal, lat=39.35, lon=2.98, target_date=date(2022, 10, 26))
print(f"Variables retrieved: {len(weather)}")
print(f"AEMET: {len([k for k in weather if k.startswith('ae_')])}")
print(f"OpenMeteo: {len([k for k in weather if k.startswith('om_')])}")
print()
print("Sample values:")
for k in list(weather.keys())[:5]:
    print(f"  {k}: {weather[k]:.2f}")

In [ ]:
weather = get_all_weather(df_bal, lat=39.35, lon=2.98, target_date=date(2022, 10, 26))

# Access values
temp_aemet = weather.get('ae_ta')
temp_openmeteo = weather.get('om_temperature_2m')
uv = weather.get('om_uv_index')
humidity = weather.get('ae_hr')

print(weather)